In [1]:
import grid
import model
import pickle
import os

def newNeuralNetwork():
    return model.NeuralNetwork((70, 20), "any", 0.1, 64)

folder_name = "output/process-output_4"
if not os.path.isdir(folder_name):
    os.makedirs(folder_name)
if len(os.listdir(folder_name)) > 0:
    raise Exception("The output folder is not empty")

In [2]:
main_nn = newNeuralNetwork()
root_node = grid.generateSparseningThenSteepeningGrid(main_nn, 20, 0)

In [3]:
with open("model.py", "r") as source_file:
    model_source = source_file.read()

with open("{}/final.pickle".format(folder_name), "wb") as output_file:
    pickle.dump({"root": root_node, "model_source": model_source}, output_file)